In [43]:
# Import library
import time
import datetime
from kafka3 import KafkaProducer
import pandas as pd
import pymongo
from pymongo import MongoClient
import re
import json

In [44]:
# Load climate data from file
climate_streaming = pd.read_csv("../Datasets/climate_streaming.csv")

In [45]:
# Change the ip address to host ip address
ip_address = "192.168.224.1"

In [46]:
# Get Latest Climate Date
client = MongoClient(ip_address, 27017) 
db = client['A3_db'] # Using A3_db
fire_collection = db['fire_historic'] # Using collection fire_historic
response = fire_collection.find({},{'date':1}).sort('date',-1).limit(1) # Get latest document
latest_climate = response[0]['date']

In [47]:
# Map function to value, if error is raised return None
# If value is classified as null also return None
def val_or_null(value, func, null=None):
    try:
        value = func(value)
        if value == null:
            return None
        else:
            return value
    except:
        return None

In [48]:
def publish_message(producer_instance, topic_name, message):
    try:
        data = message.encode('utf-8')
        producer_instance.send(topic_name, data)
        producer_instance.flush()
        print('Message published successfully. Data:\n ' + str(data) + '\n')
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [49]:
producer = KafkaProducer(bootstrap_servers=[f'{ip_address}:9092'],
                                  api_version=(0, 10))

created_date = latest_climate

# Iterate over the climate data and send messages to Kafka topic
while True:
    # Select row with replacement
    row = climate_streaming.sample(n=1, replace=True) 
    
    # Increment created_date by 1 day
    # Store as string first, but later in consumer we will change the format
    created_date = pd.to_datetime(created_date) + pd.DateOffset(days=1)
    created_date = created_date.strftime("%Y/%m/%d")
    
    # Get the precipitation value and flag
    precipitation = str(row['precipitation '].values[0]).strip()
    p = re.match(r'([0-9.]+)([a-z]+)', precipitation, re.I)
    if p:
        p = p.groups()
        precipitation_flag = str(p[1])
        precipitation = float(p[0])
    # If missing set, flag to 'o' and value to 99.99
    else:
        precipitation_flag = 'o'
        precipitation = 99.99
    
    # Get the value for everything. Get null if value is invalid
    latitude = val_or_null(row.latitude, float)
    longitude = val_or_null(row.longitude, float)
    air_temperature = val_or_null(row.air_temperature_celcius, float)
    relative_humidity = val_or_null(row.relative_humidity, float)
    windspeed_knots = val_or_null(row.windspeed_knots, float)
    max_wind_speed = val_or_null(row.max_wind_speed, float)
    GHI = val_or_null(row['GHI_w/m2'], float)
    
    # Produce the document
    document = {
        'date' : created_date,
        'latitude': latitude,
        'longitude': longitude,
        'air_temperature': air_temperature,
        'relative_humidity': relative_humidity,
        'windspeed_knots': windspeed_knots,
        'max_wind_speed': max_wind_speed,
        'precipitation_flag': precipitation_flag,
        'precipitation': precipitation,
        'GHI' : GHI,
        'producer': "Producer1"
    }
    
    # Convert dict (document) into json
    message = json.dumps(document)
    
    # Send message to Kafka topic
    publish_message(producer, "climate_topic", message)

    # Sleep for 10 seconds (1 day)
    time.sleep(10)
    

Message published successfully. Data:
 b'{"date": "2023/01/02", "latitude": -37.8276, "longitude": 142.7573, "air_temperature": 9.0, "relative_humidity": 42.1, "windspeed_knots": 7.6, "max_wind_speed": 12.0, "precipitation_flag": "G", "precipitation": 0.01, "GHI": 82.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/01/03", "latitude": -36.8835, "longitude": 142.2098, "air_temperature": 11.0, "relative_humidity": 41.6, "windspeed_knots": 7.9, "max_wind_speed": 15.0, "precipitation_flag": "G", "precipitation": 0.01, "GHI": 101.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/01/04", "latitude": -36.2693, "longitude": 143.1768, "air_temperature": 12.0, "relative_humidity": 46.6, "windspeed_knots": 6.7, "max_wind_speed": 14.0, "precipitation_flag": "G", "precipitation": 0.0, "GHI": 106.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/01/05", "latitude": -36.8741, "longitude": 143.

Message published successfully. Data:
 b'{"date": "2023/01/30", "latitude": -37.399, "longitude": 148.081, "air_temperature": 14.0, "relative_humidity": 49.9, "windspeed_knots": 13.7, "max_wind_speed": 22.0, "precipitation_flag": "G", "precipitation": 0.03, "GHI": 120.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/01/31", "latitude": -35.892, "longitude": 145.62, "air_temperature": 10.0, "relative_humidity": 42.6, "windspeed_knots": 8.9, "max_wind_speed": 15.0, "precipitation_flag": "G", "precipitation": 0.04, "GHI": 91.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/02/01", "latitude": -37.603, "longitude": 149.324, "air_temperature": 16.0, "relative_humidity": 53.6, "windspeed_knots": 12.1, "max_wind_speed": 16.9, "precipitation_flag": "G", "precipitation": 0.16, "GHI": 133.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/02/02", "latitude": -37.437, "longitude": 148.104,

Message published successfully. Data:
 b'{"date": "2023/02/27", "latitude": -36.009, "longitude": 143.185, "air_temperature": 28.0, "relative_humidity": 48.3, "windspeed_knots": 11.9, "max_wind_speed": 22.0, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 243.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/02/28", "latitude": -38.16, "longitude": 143.803, "air_temperature": 20.0, "relative_humidity": 62.2, "windspeed_knots": 7.1, "max_wind_speed": 15.0, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 154.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/03/01", "latitude": -35.2881, "longitude": 142.5679, "air_temperature": 17.0, "relative_humidity": 38.7, "windspeed_knots": 16.8, "max_wind_speed": 22.9, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 159.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/03/02", "latitude": -36.1858, "longitude": 143.885

Message published successfully. Data:
 b'{"date": "2023/03/27", "latitude": -35.957, "longitude": 141.088, "air_temperature": 9.0, "relative_humidity": 42.4, "windspeed_knots": 7.2, "max_wind_speed": 9.9, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 82.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/03/28", "latitude": -37.3847, "longitude": 142.8935, "air_temperature": 20.0, "relative_humidity": 43.6, "windspeed_knots": 10.3, "max_wind_speed": 15.0, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 181.0, "producer": "Producer1"}'

Message published successfully. Data:
 b'{"date": "2023/03/29", "latitude": -37.976, "longitude": 145.649, "air_temperature": 22.0, "relative_humidity": 62.0, "windspeed_knots": 7.9, "max_wind_speed": 11.1, "precipitation_flag": "I", "precipitation": 0.0, "GHI": 170.0, "producer": "Producer1"}'



KeyboardInterrupt: 